# Building Agent with LangGraph

This notebook explores the capabilities of LangGraph for building intelligent agents. We will start with a very simple graph to illustrate the fundamental concepts of states, nodes, and edges. Then, we will progress to building a more complex agent that can perform tasks, use tools, and even incorporate human feedback. Finally, we will construct an advanced essay writing agent that leverages multiple components for planning, research, generation, and reflection.

In [ ]:
!pip install tavily-python

## The Simplest Graph

Let's build a simple graph with 3 nodes and one conditional edge. 

This initial example demonstrates the core components of a LangGraph application: defining a state, creating nodes as Python functions that modify this state, and connecting these nodes with edges to define the flow of execution. A conditional edge is introduced to show how the graph can make decisions based on the current state.

![Screenshot 2024-08-20 at 3.11.22 PM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dba5f465f6e9a2482ad935_simple-graph1.png)

### State

First, define the [State](https://langchain-ai.github.io/langgraph/concepts/low_level/#state) of the graph. 

The State schema serves as the input schema for all Nodes and Edges in the graph.

It can be defined using any Python type, but is typically a TypedDict or Pydantic BaseModel.
Let's use the `BaseModel` class here.

In [ ]:
from pydantic import BaseModel


class State(BaseModel):
    graph_state: str

### Nodes

[Nodes](https://langchain-ai.github.io/langgraph/concepts/low_level/#nodes) are just python functions.

The first positional argument is the state, as defined above.

Because the state is a `TypedDict` with schema as defined above, each node can access the key, `graph_state`, with `state['graph_state']`.

Each node returns a new value of the state key `graph_state`.
  
By default, the new value returned by each node [will override](https://langchain-ai.github.io/langgraph/concepts/low_level/#reducers) the prior state value.

In [ ]:
def node_1(state: State):
    print("---Node 1---")
    return {"graph_state": state.graph_state + " I am"}


def node_2(state: State):
    print("---Node 2---")
    return {"graph_state": state.graph_state + " happy!"}


def node_3(state: State):
    print("---Node 3---")
    return {"graph_state": state.graph_state + " sad!"}

In [ ]:
from IPython.display import Image, display
from langgraph.graph import END, START, StateGraph

# Build graph
builder = StateGraph(State)
builder.add_node("node_1", node_1)
builder.add_node("node_2", node_2)
builder.add_node("node_3", node_3)

### Edges

[Edges](https://langchain-ai.github.io/langgraph/concepts/low_level/#edges) connect the nodes.

Normal Edges are used if you want to *always* go from, for example, `node_1` to `node_2`.

[Conditional Edges](https://langchain-ai.github.io/langgraph/concepts/low_level/#conditional-edges) are used if you want to *optionally* route between nodes.
 
Conditional edges are implemented as functions that return the next node to visit based upon some logic.

In [ ]:
import random
from typing import Literal


def decide_mood(state) -> Literal["node_2", "node_3"]:

    # Often, we will use state to decide on the next node to visit
    user_input = state.graph_state

    # Here, let's just do a 50 / 50 split between nodes 2, 3
    if random.random() < 0.5:

        # 50% of the time, we return Node 2
        return "node_2"

    # 50% of the time, we return Node 3
    return "node_3"

In [ ]:
# Logic
builder.add_edge(START, "node_1")
builder.add_conditional_edges("node_1", decide_mood)
builder.add_edge("node_2", END)
builder.add_edge("node_3", END)

In [ ]:
graph = builder.compile()

display(Image(graph.get_graph().draw_mermaid_png()))

### Graph Invocation

The compiled graph implements the [runnable](https://python.langchain.com/docs/concepts/runnables/) protocol.

This provides a standard way to execute LangChain components.
 
`invoke` is one of the standard methods in this interface.

The input is a dictionary `{"graph_state": "Hi, this is lance."}`, which sets the initial value for our graph state dict.

When `invoke` is called, the graph starts execution from the `START` node.

It progresses through the defined nodes (`node_1`, `node_2`, `node_3`) in order.

The conditional edge will traverse from node `1` to node `2` or `3` using a 50/50 decision rule.

Each node function receives the current state and returns a new value, which overrides the graph state.

The execution continues until it reaches the `END` node.

In [ ]:
graph.invoke({"graph_state": ""})

## Simple Agent with Function Calling tool

Now, we can extend this into a generic agent architecture.

This section demonstrates how to build a basic agent capable of using tools through function calling. We will define a set of tools (e.g., for arithmetic operations) and bind them to a language model. The LangGraph setup will then manage the flow: the agent (LLM) decides if a tool is needed, calls the appropriate tool, and then potentially loops back to the agent for further processing or produces a final response. This illustrates a fundamental ReAct (Reasoning and Acting) style loop.

In the function calling notebook, we explored invoked the model and, if it chose to call a tool, we returned a `ToolMessage` to the user.
 
![Screenshot 2024-08-21 at 12.45.43 PM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbac0b4a2c1e5e02f3e78b_agent2.png)

In [ ]:
from langchain_google_vertexai import ChatVertexAI, VertexAI, VertexAIEmbeddings

llm = ChatVertexAI(model="gemini-2.0-flash")

In [ ]:
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b


# This will be a tool


def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


def divide(a: int, b: int) -> float:
    """Divide a and b.

    Args:
        a: first int
        b: second int
    """
    return a / b


tools = [add, multiply, divide]

llm_with_tools = llm.bind_tools(tools, parallel_tool_calls=False)

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import MessagesState

# System message
sys_msg = SystemMessage(
    content="You are a helpful assistant tasked with performing arithmetic on a set of inputs."
)

# Node


def assistant(state: MessagesState):
    return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

For state, we use a prebuilt `MessagesState` which has 'messages' property, and define a `Tools` node with our list of tools.

The `assistant` node is just our model with bound tools.

We create a graph with `assistant` and `tools` nodes.

We add `tools_condition` edge, which routes to `End` or to `tools` based on  whether the `assistant` calls a tool.

Now, we add one new step:

We connect the `Tools` node *back* to the `Assistant`, forming a loop.

* After the `assistant` node executes, `tools_condition` checks if the model's output is a tool call.
* If it is a tool call, the flow is directed to the `tools` node.
* The `tools` node connects back to `assistant`.
* This loop continues as long as the model decides to call tools.
* If the model response is not a tool call, the flow is directed to END, terminating the process.

In [ ]:
from IPython.display import Image, display
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import ToolNode, tools_condition

builder = StateGraph(MessagesState)

# Define nodes: these do the work
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges: these determine how the control flow moves
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition,
)
builder.add_edge("tools", "assistant")

In [ ]:
react_graph = builder.compile()

# Show
display(Image(react_graph.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
initial_input = {"messages": HumanMessage(content="Multiply 2 and 3.")}

# thread = {"configurable": {"thread_id": "1"}}

for event in react_graph.stream(initial_input, stream_mode="values"):
    event["messages"][-1].pretty_print()

## Breakpoint and Human-in-the-loop

LangGraph allows for the inclusion of breakpoints in the agent's execution flow. This is crucial for debugging, monitoring, and enabling human-in-the-loop interventions. By interrupting the graph at a specific node (e.g., before a tool is called), we can inspect the state and decide whether to proceed, modify the state, or halt execution. This section demonstrates how to set up such a breakpoint and prompt for user approval before continuing.

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

react_graph = builder.compile(checkpointer=memory, interrupt_before=["tools"])

thread = {"configurable": {"thread_id": "1"}}

for event in react_graph.stream(initial_input, thread, stream_mode="values"):
    event["messages"][-1].pretty_print()

In [ ]:
# Thread
thread = {"configurable": {"thread_id": "2"}}

# Run the graph until the first interruption
for event in react_graph.stream(initial_input, thread, stream_mode="values"):
    event["messages"][-1].pretty_print()

# Get user feedback
user_approval = input("Do you want to call the tool? (yes/no): ")

# Check approval
if user_approval.lower() == "yes":

    # If approved, continue the graph execution
    for event in react_graph.stream(None, thread, stream_mode="values"):
        event["messages"][-1].pretty_print()

else:
    print("Operation cancelled by user.")

---
## Build a complex Essays writing agent

This section demonstrates how to build a LangGraph-powered AI agent to generate, revise, and critique essays using Gemini. The LangGraph code was adapted from the awesome DeepLearning.AI course on AI Agents in LangGraph.

By defining a structured state flow with nodes such as "Planner," "Research Plan," "Generate," "Reflect," and "Research Critique," the system iteratively creates an essay on a given topic, incorporates feedback, and provides research-backed insights.

The agent will perform the following steps:
1.  **Plan:** Create an initial outline for the essay based on the given topic.
2.  **Research Plan:** Generate search queries based on the topic to gather initial information.
3.  **Generate:** Write a draft of the essay using the plan and researched content.
4.  **Reflect:** Critique the generated draft, identifying areas for improvement.
5.  **Research Critique:** Generate new search queries based on the critique to find information for revisions.
6.  **Loop:** The generation, reflection, and research critique steps can loop a specified number of times to iteratively improve the essay.

<img width="1084" alt="image" src="https://camo.githubusercontent.com/f39ea212d055cb3982e931286d62cd08ec812881f4257ff1a4fd0755fc9cb628/68747470733a2f2f6769746875622e636f6d2f476f6f676c65436c6f7564506c6174666f726d2f67656e657261746976652d61692f626c6f622f6d61696e2f776f726b73686f70732f61692d6167656e74732f332d6c616e6767726170682d65737361792e706e673f7261773d31">

The workflow enables automated essay generation with revision controls, making it ideal for structured writing tasks or educational use cases. Additionally, the notebook uses external search tools to gather and integrate real-time information into the essay content.



In [ ]:
import os
from typing import List, Optional

from IPython.display import Markdown, display

### Configure Tavily
Get an API key for [Tavily](https://tavily.com/), a web search API for Generative AI models.
Tavily will be used by our agent to perform research and gather up-to-date information from the web to incorporate into the essay.

In [ ]:
from tavily import TavilyClient

os.environ["TAVILY_API_KEY"] = "YOUR API KEY"
tavily = TavilyClient()

### Initialize agent memory, agent state, and schema for search queries

Before defining the agent's nodes and graph structure, we need to set up the foundational components:
* **Agent State (`AgentState`):** This pydantic `BaseModel` will hold all the information that flows through our graph. It includes the task, essay plan, draft, critique, researched content, revision number, and maximum allowed revisions. Each node in the graph will read from and write to this state.
* **Search Queries Schema (`Queries`):** This Pydantic `BaseModel` defines the expected structure for the search queries that our research nodes will generate. Using a schema helps ensure that the LLM produces queries in a usable format.

In [ ]:
class AgentState(BaseModel):
    task: str
    plan: Optional[str] = ""
    draft: Optional[str] = ""
    critique: Optional[str] = ""
    content: List[str]
    revision_number: int
    max_revisions: int


class Queries(BaseModel):
    """Variants of query to search for"""

    queries: list[str]

### Define prompt templates for each stage

Clear and effective prompts are essential for guiding the LLM at each stage of the essay writing process. We define specific prompt templates for:
* **PLAN_PROMPT:** Instructs the LLM to act as an expert writer and create a high-level outline for the essay.
* **RESEARCH_PLAN_PROMPT:** Guides the LLM to act as a researcher and generate search queries to gather initial information for the essay.
* **WRITER_PROMPT:** Instructs the LLM to act as an essay assistant, write a 3-page essay based on the plan and research, and revise based on critique. It also specifies Markdown formatting.
* **REFLECTION_PROMPT:** Tells the LLM to act as a teacher grading the essay, providing critique and detailed recommendations.
* **RESEARCH_CRITIQUE_PROMPT:** Guides the LLM to generate search queries based on the essay draft and the critique to find information for revisions.

In [ ]:
PLAN_PROMPT = """You are an expert writer tasked with writing a high level outline of an essay.
Write such an outline for the user provided topic. Give an outline of the essay along with any
relevant notes or instructions for the sections."""

RESEARCH_PLAN_PROMPT = """You are a researcher charged with providing information that can
be used when writing the following essay. Generate a list of search queries that will gather
any relevant information. Only generate 3 queries max."""

WRITER_PROMPT = """You are an essay assistant tasked with writing excellent 3-pages essays.
Generate the best essay possible for the user's request and the initial outline.
If the user provides critique, respond with a revised version of your previous attempts.
Use Markdown formatting to specify a title and section headers for each paragraph.
Utilize all of the information below as needed:
---
{content}"""

REFLECTION_PROMPT = """You are a teacher grading an essay submission.
Generate critique and recommendations for the user's submission.
Provide detailed recommendations, including requests for length, depth, style, etc."""

RESEARCH_CRITIQUE_PROMPT = """You are a researcher charged with providing information that can
be used when making any requested revisions for the draft (draft and requests are outlined below).
Generate a list of search queries that will gather any relevant information.
Only generate 3 queries max."""

### Define node functions for each stage

Each node in our LangGraph represents a specific step in the essay writing workflow. These Python functions take the current `AgentState` as input and return a dictionary with the updated state values for their respective operations:

* `plan_node`: Invokes the LLM with the `PLAN_PROMPT` to generate an essay outline.
* `research_plan_node`: Uses the LLM (with structured output for `Queries`) and the `RESEARCH_PLAN_PROMPT` to generate search queries. It then executes these queries using the Tavily client and appends the results to the state's content list.
* `generation_node`: Invokes the LLM with the `WRITER_PROMPT`, the essay task, the plan, and the researched content to generate an essay draft. It also increments the revision number.
* `reflection_node`: Uses the LLM with the `REFLECTION_PROMPT` to critique the current draft.
* `research_critique_node`: Similar to `research_plan_node`, but uses the `RESEARCH_CRITIQUE_PROMPT` along with the current draft and critique to generate targeted search queries for revision. Results are added to the content list.
* `should_continue`: This is a conditional edge function. It checks if the current `revision_number` has exceeded `max_revisions`. If so, it returns `END` to terminate the graph; otherwise, it returns "reflect" to continue the revision cycle.

In [ ]:
def plan_node(state: AgentState):
    messages = [
        SystemMessage(content=PLAN_PROMPT),
        HumanMessage(content=state.task),
    ]
    response = llm.invoke(messages)
    return {"plan": response.content}


# Conducts research based on the generated plan and web search results
def research_plan_node(state: AgentState):
    queries = llm.with_structured_output(Queries).invoke(
        [
            SystemMessage(content=RESEARCH_PLAN_PROMPT),
            HumanMessage(content=state.task),
        ]
    )
    content = state.content or []
    for q in queries.queries:
        response = tavily.search(query=q, max_results=3)
        for r in response["results"]:
            content.append(r["content"])
    return {"content": content}


# Generates a draft based on the content and plan
def generation_node(state: AgentState):
    content = "\n\n".join(state.content or [])
    user_message = HumanMessage(
        content=f"{state.task}\n\nHere is my plan:\n\n{state.plan}"
    )
    messages = [
        SystemMessage(content=WRITER_PROMPT.format(content=content)),
        user_message,
    ]
    response = llm.invoke(messages)
    return {
        "draft": response.content,
        "revision_number": state.revision_number + 1,
    }


# Provides feedback or critique on the draft
def reflection_node(state: AgentState):
    messages = [
        SystemMessage(content=REFLECTION_PROMPT),
        HumanMessage(content=state.draft),
    ]
    response = llm.invoke(messages)
    return {"critique": response.content}


# Conducts research based on the critique
def research_critique_node(state: AgentState):
    queries = llm.with_structured_output(Queries).invoke(
        [
            SystemMessage(content=RESEARCH_CRITIQUE_PROMPT),
            HumanMessage(content=state.draft),
            HumanMessage(content=state.critique),
        ]
    )
    content = state.content or []
    for q in queries.queries:
        response = tavily.search(query=q, max_results=3)
        for r in response["results"]:
            content.append(r["content"])
    return {"content": content}


# Determines whether the critique and research cycle should
# continue based on the number of revisions
def should_continue(state):
    if state.revision_number > state.max_revisions:
        return END
    return "reflect"

### Define and compile the graph

With the state, prompts, and node functions defined, we can now construct the essay writing agent's graph:
1.  **Initialize `StateGraph`:** We create an instance of `StateGraph` with our `AgentState`.
2.  **Add Nodes:** Each function defined in the previous step (`plan_node`, `generation_node`, etc.) is added as a node to the graph, with a unique string identifier.
3.  **Set Entry Point:** `builder.set_entry_point("planner")` designates the `planner` node as the starting point of the graph.
4.  **Add Edges:**
    * Sequential Edges: We define the primary flow: `planner` -> `research_plan` -> `generate`. And the revision loop: `reflect` -> `research_critique` -> `generate`.
    * Conditional Edges: `builder.add_conditional_edges` is used after the `generate` node. It calls the `should_continue` function. Based on its return value, the graph either transitions to the `reflect` node (to continue revising) or to `END` (to finish).
5.  **Initialize Memory:** A `MemorySaver` is initialized to enable checkpointing, which saves the state of the graph.
6.  **Compile Graph:** `builder.compile(checkpointer=memory)` finalizes the graph structure and incorporates the memory saver, making the graph ready for execution.

In [ ]:
# Initialize the state graph
builder = StateGraph(AgentState)

# Add nodes for each step in the workflow
builder.add_node("planner", plan_node)
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.add_node("research_plan", research_plan_node)
builder.add_node("research_critique", research_critique_node)

# Set the entry point of the workflow
builder.set_entry_point("planner")

# Add conditional edges for task continuation or end
builder.add_conditional_edges(
    "generate", should_continue, {END: END, "reflect": "reflect"}
)

# Define task sequence edges
builder.add_edge("planner", "research_plan")
builder.add_edge("research_plan", "generate")

builder.add_edge("reflect", "research_critique")
builder.add_edge("research_critique", "generate")

# Initialize agent memory
memory = MemorySaver()

# Compile the graph with memory state management
graph = builder.compile(checkpointer=memory)

In [ ]:
Image(graph.get_graph().draw_mermaid_png())

### Run the agent - write on!

Now it's time to run our essay writing agent. We'll provide an initial essay topic and configuration, then stream the agent's execution to observe each step and its output.

In [ ]:
# Define the topic of the essay
ESSAY_TOPIC = (
    "What were the impacts of Hurricane Helene and Hurricane Milton in 2024?"
)

# Define a thread configuration with a unique thread ID
thread = {"configurable": {"thread_id": "1"}}

# Stream through the graph execution with an initial task and state
for s in graph.stream(
    {
        "task": ESSAY_TOPIC,  # Initial task
        "max_revisions": 2,  # Maximum number of revisions allowed
        "revision_number": 1,  # Current revision number
        "content": [],  # Initial empty content list
    },
    thread,
):
    step = next(iter(s))
    display(Markdown(f"# {step}"))
    for key, content in s[step].items():
        if key == "revision_number":
            display(Markdown(f"**Revision Number**: {content}"))
        elif isinstance(content, list):
            for c in content:
                display(Markdown(c))
        else:
            display(Markdown(content))
    print("\n---\n")

Copyright 2025 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License